In [38]:
import sys
sys.path.append('../')

import numpy as np

from ForwardModeling.ForwardProcessing1D import forward_with_trace_calcing
from Inversion.Strategies.SeismDiffInversion1D import inverse
from Inversion.Optimizators.Optimizations import LBFGSBOptimization, DifferentialEvolution, TrustKrylov, TrustConstr, \
    ConjugateGradient
from Tests.test_ForwardProcessing1D import get_model_2layered, get_model_3layered
from Data.geol_models import get_model_3layered_fluid_rp_dicts
from Objects.Data.WavePlaceholder import OWT, WaveDataPlaceholder
from Objects.Models.Models import SeismicModel1D
from Objects.Attributes.RockPhysics.RockPhysicsAttribute import RockPhysicsAttribute
from Objects.Models.Layer1D import Layer1D, LayerOPT
import time

from matplotlib import pyplot as plt
from Inversion.Strategies.SeismDiffInversion1D import func_to_optimize

ImportError: cannot import name 'get_model_3layered_fluid_rp_dicts' from 'Data.geol_models' (/home/apenkin/workspace/rpsi_proj/rpsi/Data/geol_models.py)

In [30]:
def plot_1D_err(forward_params, nvals, min_max=None, dot=False, vline_x=None):
    
    if min_max is None:
        min_val = forward_params['model'].get_optimization_option("min", vectorize=True)
        max_val = forward_params['model'].get_optimization_option("max", vectorize=True)
        
    else:
        min_val = np.array([min_max[0]])
        max_val = np.array([min_max[1]])
    
    dval = (max_val - min_val) / nvals

    val_x = [min_val + dval*i for i in range(nvals+1)]
    errs = []
    for val in val_x:
        errs.append(func_to_optimize(val, placeholders, forward_params, helper=None, show_tol=False))
        
    plt.plot(val_x, errs)
    if dot:
        plt.plot(val_x, errs, 'o')
        
    if vline_x is not None:
        plt.vlines(vline_x, ymin=min(errs), ymax=max(errs), colors='r')
        
    plt.show()

def optimization_func_2D(forward_params, x, y):
    indexes_1 = np.arange(x.shape[0])
    indexes_2 = np.arange(x.shape[1])
    
    Z = np.zeros((x.shape[0], x.shape[1]))
    
    for ind1 in indexes_1:
        for ind2 in indexes_2:
            Z[ind1, ind2] = func_to_optimize([x[ind1, ind2], y[ind1, ind2]], placeholders, forward_params, helper=None, show_tol=False)
        
    return Z

def plot_2D_err(forward_params, nvals, ncontous=10, points=None):
    min_val = forward_params['model'].get_optimization_option("min", vectorize=True)
    max_val = forward_params['model'].get_optimization_option("max", vectorize=True)
    
    dval = (max_val - min_val) / nvals

    val_1_x = [min_val[0] + dval[0]*i for i in range(nvals[0]+1)]
    val_2_x = [min_val[1] + dval[1]*i for i in range(nvals[1]+1)]

    X, Y = np.meshgrid(val_1_x, val_2_x)

    Z = optimization_func_2D(forward_params, X, Y)
    
    plt.contourf(X, Y, Z, ncontous, cmap='seismic')
    plt.colorbar()
    
    if points is not None:
        plt.plot(points[0], points[1], marker='x', color='r')

# 2-layered model

In [20]:
layer_1 = Layer1D(h,
                  rp_attribute=RockPhysicsAttribute(layer_1_dict["components"], layer_1_dict["name"]),
                  seism_attribute=None,
                  opt=LayerOPT.RP)

layer_2 = Layer1D(-1,
                  rp_attribute=RockPhysicsAttribute(layer_2_dict["components"], layer_2_dict["name"]),
                  seism_attribute=None,
                  opt=LayerOPT.RP)

NameError: name 'h' is not defined

In [ ]:
dx = 100
nx = 20
x_rec = [i * dx for i in range(1, nx)]
wave_types = [OWT.PdPu]

In [ ]:
model = SeismicModel1D([layer_1, layer_2])
model_true = SeismicModel1D([layer_1, layer_2])
model_opt = SeismicModel1D([layer_1, layer_2])

In [ ]:
observe, test_seismic = \
    forward_with_trace_calcing(model, x_rec,
                               dt=3e-03, trace_len=1500, wavetypes=wave_types, display_stat=True,
        visualize_res=False)

In [ ]:
forward_params = {
    "model": model,
    "x_rec": x_rec,
    "dt": 3e-03,
    "trace_len": 1500,
    "wavetypes": wave_types,
    "display_stat": False,
    "visualize_res": False
}

In [ ]:
placeholders = {}
for wt in wave_types:
    placeholders[wt] = WaveDataPlaceholder(
        wt,
        test_seismic[wt]["rays"],
        test_seismic[wt]["seismogram"]
    )

In [ ]:
optimizers = [
    LBFGSBOptimization(
        maxiter=15000,
        maxfun=15000,
        factr=10000,
        maxls=50,
        epsilon=0.000001
    )
]

In [ ]:
optimizers = [
    DifferentialEvolution()
]

In [ ]:
forward_params['model'].layers[0]['Km'] = 10

In [ ]:
forward_params['model'].layers[1]['Km'] = 25

In [ ]:
forward_params['model'].layers[0].rp_attribute.vals_dict['Km']["optimize"] = True

In [ ]:
forward_params['model'].layers[1].rp_attribute.vals_dict['Km']["optimize"] = True

In [ ]:
inversed_model = inverse(optimizers, error=0.01, placeholders=placeholders, forward_params=forward_params)

In [ ]:
inversed_model

## Single variable opt

In [ ]:
from Inversion.Strategies.SeismDiffInversion1D import func_to_optimize

In [ ]:
forward_params['model'].layers[0].rp_attribute.vals_dict['Km']["optimize"] = False

In [ ]:
forward_params['model'].layers[1].rp_attribute.vals_dict['Km']["optimize"] = True

In [ ]:
forward_params['model'].layers[0]['Km'] = 7.3

In [ ]:
forward_params['model'].layers[1]['Km'] = 21.5

In [ ]:
forward_params['model'].get_optimization_option("val", vectorize=True)

In [ ]:
min_val = forward_params['model'].get_optimization_option("min", vectorize=True)
max_val = forward_params['model'].get_optimization_option("max", vectorize=True)

In [ ]:
nvals = 30

In [ ]:
dval = (max_val - min_val) / nvals

val_x = [min_val + dval*i for i in range(nvals+1)]
errs = []
for val in val_x:
    errs.append(func_to_optimize(val, placeholders, forward_params, helper=None, show_tol=False))

In [ ]:
plt.plot(val_x, errs)

## Duo variable opt

In [ ]:
forward_params['model'].layers[1].rp_attribute.vals_dict['Km']["optimize"] = True

In [ ]:
forward_params['model'].get_optimization_option("val", vectorize=True)

In [ ]:
min_val = forward_params['model'].get_optimization_option("min", vectorize=True)
max_val = forward_params['model'].get_optimization_option("max", vectorize=True)

In [ ]:
def f(x, y):
    return np.sin(x) ** 10 + np.cos(10 + y * x) * np.cos(x)


In [ ]:
dval = (max_val - min_val) / nvals

val_1_x = [min_val[0] + dval[0]*i for i in range(nvals[0]+1)]
val_2_x = [min_val[1] + dval[1]*i for i in range(nvals[1]+1)]

X, Y = np.meshgrid(val_1_x, val_2_x)

Z = optimization_func_2D(X, Y)

In [ ]:
Z.shape

In [ ]:
X.shape

In [ ]:
plt.contourf(X, Y, Z, 10, cmap='cool')
plt.colorbar()

# 3-layered model

In [31]:
h = 1000

In [32]:
layer_1_dict, layer_2_dict, layer_3_dict = get_model_3layered_fluid_rp_dicts()

In [33]:
layer_3_1 = Layer1D(h,
                  rp_attribute=RockPhysicsAttribute(layer_1_dict["components"], layer_1_dict["name"]),
                  seism_attribute=None,
                  opt=LayerOPT.RP)

layer_3_2 = Layer1D(h,
                  rp_attribute=RockPhysicsAttribute(layer_2_dict["components"], layer_2_dict["name"]),
                  seism_attribute=None,
                  opt=LayerOPT.RP)

layer_3_3= Layer1D(-1,
                  rp_attribute=RockPhysicsAttribute(layer_3_dict["components"], layer_2_dict["name"]),
                  seism_attribute=None,
                  opt=LayerOPT.RP)

In [34]:
dx = 300
nx = 5
x_rec = [i * dx for i in range(1, nx+1)]
# wave_types = [OWT.PdPu]
wave_types = [OWT.PdPu, OWT.PdSVu]

In [35]:
model_3l = SeismicModel1D([layer_3_1, layer_3_2, layer_3_3])

In [36]:
observe, test_seismic = \
    forward_with_trace_calcing(model_3l, x_rec,
                               dt=3e-03, trace_len=1500, wavetypes=wave_types, display_stat=True,
        visualize_res=False)

KeyboardInterrupt: 

In [ ]:
forward_params = {
    "model": model_3l,
    "x_rec": x_rec,
    "dt": 3e-03,
    "trace_len": 1500,
    "wavetypes": wave_types,
    "display_stat": False,
    "visualize_res": False
}

In [ ]:
placeholders = {}
for wt in wave_types:
    placeholders[wt] = WaveDataPlaceholder(
        wt,
        test_seismic[wt]["rays"],
        test_seismic[wt]["seismogram"]
    )

In [ ]:
optimizers_lbfgs = [
    LBFGSBOptimization(
        maxiter=15000,
        maxfun=15000,
#         factr=10000,
#         maxls=50,
#         epsilon=0.000001
    )
]

In [ ]:
optimizers_cg = [
    ConjugateGradient()
]

In [ ]:
optimizers_de = [
    DifferentialEvolution(
          popsize= 5,
          maxiter= 20000,
          init= "latinhypercube",
          strategy= "best1bin",
          disp= False,
          polish= False,
          tol= 0.00001,
          mutation= 1.5,
          recombination= 0.6,
          workers= 8
    )
]

In [ ]:
optimizers_tc = [
    TrustConstr(
        opt_options={
            "initial_tr_radius": 0.7,
            "maxiter": 1000000
#             "gtol":1e-4,
        }
    ),
#     LBFGSBOptimization(
#         maxiter=15000,
#         maxfun=15000,
#         factr=10000,
#         maxls=50,
#         epsilon=0.000001
#     )
]

### Some model changes

In [ ]:
forward_params['model'].layers[0].rp_attribute.vals_dict['phi_s']['optimize'] = False
forward_params['model'].layers[0].rp_attribute.vals_dict['Kf']['optimize'] = False
forward_params['model'].layers[0].rp_attribute.vals_dict['rho_f']['optimize'] = False
forward_params['model'].layers[0].rp_attribute.vals_dict['phi']['optimize'] = False

forward_params['model'].layers[1].rp_attribute.vals_dict['phi_s']['optimize'] = False
forward_params['model'].layers[1].rp_attribute.vals_dict['Kf']['optimize'] = False
forward_params['model'].layers[1].rp_attribute.vals_dict['rho_f']['optimize'] = False
forward_params['model'].layers[1].rp_attribute.vals_dict['phi']['optimize'] = False

### Km optimization

In [ ]:
opt_flag = True
# opt_flag = False
if opt_flag:
    forward_params['model'].layers[0]["Km"] = 9
    forward_params['model'].layers[0].opt = LayerOPT.RP
    forward_params['model'].layers[0].rp_attribute.vals_dict['Km']['optimize'] = True
    
else:
    forward_params['model'].layers[0]["Km"] = 7.3
    forward_params['model'].layers[0].opt = LayerOPT.NO
    forward_params['model'].layers[0].rp_attribute.vals_dict['Km']['optimize'] = False

opt_flag = True
# opt_flag = False

if opt_flag:
    forward_params['model'].layers[1]["Km"] = 25
    forward_params['model'].layers[1].opt = LayerOPT.RP
    forward_params['model'].layers[1].rp_attribute.vals_dict['Km']['optimize'] = True

else:
    # TRUE VALUE
    forward_params['model'].layers[1]["Km"] = 21.5
    forward_params['model'].layers[1].opt = LayerOPT.NO
    forward_params['model'].layers[1].rp_attribute.vals_dict['Km']['optimize'] = False



# opt_flag = True
opt_flag = False

if opt_flag:
    forward_params['model'].layers[2]["Km"] = 25
    forward_params['model'].layers[2].opt = LayerOPT.RP
    forward_params['model'].layers[2].rp_attribute.vals_dict['Km']['optimize'] = True

else:
    # TRUE VALUE
    forward_params['model'].layers[2]["Km"] = 22
    forward_params['model'].layers[2].opt = LayerOPT.NO
    forward_params['model'].layers[2].rp_attribute.vals_dict['Km']['optimize'] = False



### Gm optimization

In [21]:
# opt_flag = True
opt_flag = False
if opt_flag:
    forward_params['model'].layers[0]["Gm"] = 5
    forward_params['model'].layers[0].opt = LayerOPT.RP
    forward_params['model'].layers[0].rp_attribute.vals_dict['Gm']['optimize'] = True
    
else:
    forward_params['model'].layers[0]["Gm"] = 2.71
    forward_params['model'].layers[0].opt = LayerOPT.NO
    forward_params['model'].layers[0].rp_attribute.vals_dict['Gm']['optimize'] = False

# opt_flag = True
opt_flag = False

if opt_flag:
    forward_params['model'].layers[1]["Gm"] = 15
    forward_params['model'].layers[1].opt = LayerOPT.RP
    forward_params['model'].layers[1].rp_attribute.vals_dict['Gm']['optimize'] = True

else:
    # TRUE VALUE
    forward_params['model'].layers[1]["Gm"] = 17.5
    forward_params['model'].layers[1].opt = LayerOPT.NO
    forward_params['model'].layers[1].rp_attribute.vals_dict['Gm']['optimize'] = False



# opt_flag = True
opt_flag = False

if opt_flag:
    forward_params['model'].layers[2]["Gm"] = 15
    forward_params['model'].layers[2].opt = LayerOPT.RP
    forward_params['model'].layers[2].rp_attribute.vals_dict['Gm']['optimize'] = True

else:
    # TRUE VALUE
    forward_params['model'].layers[2]["Gm"] = 10.7
    forward_params['model'].layers[2].opt = LayerOPT.NO
    forward_params['model'].layers[2].rp_attribute.vals_dict['Gm']['optimize'] = False



NameError: name 'forward_params' is not defined

## Model calcing

In [ ]:
forward_params['model'].get_optimization_option('val')

In [ ]:
# scale="minmax"
scale=None
start_time = time.time()
# optimizers_tc[0].opt_options["finite_diff_rel_step"] = 0.1
inversed_model = inverse(optimizers_tc, error=0.07, placeholders=placeholders, forward_params=forward_params, scale=scale)
print(f"TC time: {time.time() - start_time}")

In [ ]:
# scale="minmax"
scale=None
start_time = time.time()
inversed_model = inverse(optimizers_de, error=0.01, placeholders=placeholders, forward_params=forward_params, scale=scale)
print(f"DE time: {time.time() - start_time}")

In [ ]:
forward_params['model'].get_optimization_option("val", vectorize=True)

In [ ]:
inversed_model

In [ ]:
plot_1D_err(forward_params, 100, vline_x=inversed_model)

In [ ]:
plot_2D_err(forward_params, [5, 5], 20, points=[[inversed_model[0]], [inversed_model[1]]])

In [ ]:
res = forward_with_trace_calcing(forward_params['model'], forward_params['x_rec'], 
                           dt=forward_params['dt'], 
                           trace_len=forward_params['trace_len'], 
                           wavetypes=forward_params['wavetypes'],
            display_stat=True, visualize_res=False,
                               visualize_seismograms=True
            )

In [ ]:
inversed_model